In [1]:
#
# Import US and EM total return data (USD) matching the time period of Page:
# - US stocks: Wilshire 5000 Total Market Index, with returns from January 1971 to May 2011
# - EM stocks: MSCI Emerging Markets Index (Total Return Index), with returns from January 1988 to May 2011
#

import pandas as pd

# US
us = pd.read_csv("WILL5000PRFC_M.csv",index_col=0,sep=",")
us.index = pd.to_datetime(us.index)
usReturns = us.pct_change().dropna().loc["1971-01-31":"2011-05-31"]

# EM
em = pd.read_csv("EM.csv",index_col=0,sep=";")
em.index = pd.to_datetime(em.index)
em['EM'] = em['EM'].str.replace(',','')
em['EM'] = pd.to_numeric(em['EM'])
emReturns = em.pct_change().dropna().loc["1988-01-29":"2011-05-31"]

In [2]:
# 
# Isolate the "true" returns of EM from January 1988 to January 1998 (1O years + 1 month)
#

import numpy as np
import scipy
from scipy import stats

emReturnsTruth = emReturns.loc["1988-01-29":"1998-01-30"]
emReturnsTruth.columns = ["EM (January 1988 to January 1998)"]

emReturnsTruth.apply(lambda c: pd.Series({'Number of returns': len(c),
                                          'Mean': np.mean(c),
                                          'Variance': np.var(c, ddof=1),
                                          'Skewness': scipy.stats.skew(c, bias = False),
                                          'Kurtosis': scipy.stats.kurtosis(c, bias = False, fisher = False),
                                         }))

,EM (January 1988 to January 1998)
Number of returns,121.000000
Mean,0.015149
Variance,0.003887
Skewness,-0.251414
Kurtosis,3.828801


In [3]:
# 
# Define the interface with Portfolio Optimizer Web API
#

import requests
import json

#
def backfillReturns(returns, gaussianNoise = False, portfolioOptimizerApiKey = None):
    """
    Backfill asset returns.

    Input
    -----
  
    returns: list of lists, the asset returns
    gaussianNoise: boolean, whether to use Gaussian noise or "fat-tails" noise to backfill asset returns
    portfolioOptimizerApiKey: string, the Portfolio Optimizer API key 

    Output
    ------
    list of lists, the asset returns
    """
    # Build the input to Portfolio Optimizer API
    nb_assets = len(returns)
    if gaussianNoise:
        poRequestBody = { 'assets': list(map(lambda x: {'assetReturns': x}, returns)), 'assetsReturnsBackfillingMethod': 'conditionalSampling' }
    else:
        poRequestBody = { 'assets': list(map(lambda x: {'assetReturns': x}, returns)) }
        
    # Call the Portfolio Optimizer API 
    response = requests.post('https://api.portfoliooptimizer.io/v1/assets/returns/backfilled', json=poRequestBody,  headers = {'X-API-Key': portfolioOptimizerApiKey })
    response = response.json()

    # Decode the output
    out = []
    for asset in response['assets']:
        out.append(asset['assetReturns'])

    #
    return out

In [20]:
# 
# Backfill E.M. returns, over several simulations
#

# Internal function
def runSimulations(assetReturns, emReturnsTruthLength = 121, nbSimulations = 100, gaussianNoise = True):
    #
    moments = None
    
    # Compute nbSimulations backfilled EM returns
    for i in range(nbSimulations):
        # Gaussian noise
        backfilledReturns = backfillReturns(assetReturns, gaussianNoise)
        emBackfilledReturns = backfilledReturns[1][0:emReturnsTruthLength]

        # Compute a dataframe
        mu = np.mean(emBackfilledReturns)
        sigma = np.var(emBackfilledReturns, ddof=1)
        s = scipy.stats.skew(emBackfilledReturns, bias=False)
        k = scipy.stats.kurtosis(emBackfilledReturns, bias = False, fisher = False)

        row = pd.DataFrame([[mu, sigma, s, k]], columns = ["Mean", "Variance", "Skewness", "Kurtosis"])
        moments = pd.concat([moments, pd.DataFrame(row)], ignore_index=True)

    #
    return moments

# The number of simulations to run
nbSimulations = 10000

# The length of the E.M. stocks returns ground truth
emReturnsTruthLength = len(emReturnsTruth)

# The asset returns:
# - U.S. stocks returns, over the period January 1988 to May 2011
# - E.M. stocks returns, over the period January 1998 to May 2011
assetReturns = usReturns["1988-01-29":"2011-05-31"].values.transpose().tolist() + emReturns.loc["1998-02-01":"2011-05-31"].values.transpose().tolist()

# Compute nbSimulations backfilled returns, with and without Gaussian noise
if (gaussianNoiseMoments is None):
    gaussianNoiseMoments = runSimulations(assetReturns, emReturnsTruthLength = emReturnsTruthLength, nbSimulations = nbSimulations, gaussianNoise = True)

if (fatTailsNoiseMoments is None):
    fatTailsNoiseMoments = runSimulations(assetReturns, emReturnsTruthLength = emReturnsTruthLength, nbSimulations = nbSimulations, gaussianNoise = False)

# Average the values of the moments obtained over the nbSimulations simulations
print( gaussianNoiseMoments.mean() )
print( fatTailsNoiseMoments.mean() )

Mean        0.021715
Variance    0.003644
Skewness   -0.073580
Kurtosis    3.111197
dtype: float64
Mean        0.021756
Variance    0.003646
Skewness   -0.197386
Kurtosis    3.238327
dtype: float64
